## <font color='cyan'>*** Esther Gutiérrez --- MODULO 3 EJERCICIO A2- ADAPTACIÓN PARA MÓDULO 4***</font>
### Clasificación Autismo: Intentar determinar si una persona tiene autismo o no
De la página Kaggle, he elegido el dataset con datos de autismo disponible en https://www.kaggle.com/datasets/konikarani/autismdiagnosis

En la práctia real del módulo 3 utilicé Random Forest y Regresión logística, pero para este módulo 4 voy a dejar sólo <font color='red'>Regresión logística </font> para que sea diferente del ejemplo del titanic y así utilizo otros hiperparámetros de algoritmo, configuración diferente en cloudant, etc.

<font color=#FFFF00> 1. CARGA DEL DATASET. </font>

En este caso nos proporcionan 3 ficheros
- train.csv: datos para el entrenamiento
- test.csv: datos para hacer pruebas. En este caso, el atributo objetivo (Class/ASD) no está informado.
- sample_submission.csv: resultado de someter el conjunto de test. Todos los resultados dan 1 (autismo), no parece correcto

Utilizaré entonces el archivo train.csv, subdividiéndolo en train y test para probar y medir los diferentes algoritmos. El archivo test.csv sólo lo usaré para hacer predicciones y ver qué resultado obtengo.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing


In [39]:
dfInicial = pd.read_csv('train.csv')
pd.options.display.max_columns = None
dfInicial

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,1,0,1,0,1,0,1,1,38.172746,f,?,no,no,Austria,no,6.351166,18 and more,Self,0
1,2,0,0,0,0,0,0,0,0,0,0,47.750517,m,?,no,no,India,no,2.255185,18 and more,Self,0
2,3,1,1,1,1,1,1,1,1,1,1,7.380373,m,White-European,no,yes,United States,no,14.851484,18 and more,Self,1
3,4,0,0,0,0,0,0,0,0,0,0,23.561927,f,?,no,no,United States,no,2.276617,18 and more,Self,0
4,5,0,0,0,0,0,0,0,0,0,0,43.205790,m,?,no,no,South Africa,no,-4.777286,18 and more,Self,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,0,1,0,0,0,0,0,0,1,1,16.597187,m,Hispanic,no,no,New Zealand,no,12.999501,18 and more,Self,0
796,797,0,1,1,0,0,1,0,1,1,1,20.703001,m,White-European,no,no,Cyprus,no,13.561518,18 and more,Self,0
797,798,0,0,0,0,0,0,0,0,0,0,5.711481,m,South Asian,yes,no,New Zealand,no,2.653177,18 and more,Self,0
798,799,0,0,0,0,0,0,0,0,0,0,16.414305,f,?,no,no,Canada,no,9.069342,18 and more,Self,0


<font color=#FFFF00> 3. PREPROCESADO DE DATOS. TRATAMIENTO DE NULOS. SELECCIÓN Y DESCARTE DE FEATURES. NUEVAS FEATURES. PIPELINE </font>

Según lo comentado anteriormente, voy a realizar las siguientes modificaciones en el dataset:
- Eliminaré las columnas ID, age_desc y relation, porque no me parece que aporten información para lo que queremos predecir
- Las columnas gender, jandice, autism y used_app_before (que actualmente son object) las pasaré a columnas integer con valores 0 o 1
- En la columna ethnicity, pasaré los valores 'others' a 'Others'. También voy a imputar a este valor 'Others' los nulos (valor ?)
- Crearé una nueva columna PuntuacionTest que corresponderá a la suma de los atributos A1_Score hasta A10_Score.
- Transformaré las columnas ethnicity y contry_of_res (que actualmente son object) a columnas de tipo integer, con ayuda de un LabelEncoder que directamente asignará números a los diferentes valores

In [40]:
dfTratado = dfInicial.drop(["ID", "age_desc", "relation"], axis=1, inplace=False)
dfTratado['gender'] = dfTratado['gender'].map({'m': 0, 'f': 1})
dfTratado['jaundice'] = dfTratado['jaundice'].map({'no': 0, 'yes': 1})
dfTratado['austim'] = dfTratado['austim'].map({'no': 0, 'yes': 1})
dfTratado['used_app_before'] = dfTratado['used_app_before'].map({'no': 0, 'yes': 1})
dfTratado['ethnicity'] = dfTratado['ethnicity'].replace({'others': 'Others', '?': 'Others'})
dfTratado['PuntuacionTest'] = dfTratado['A1_Score'] + dfTratado['A2_Score'] + dfTratado['A3_Score'] + dfTratado['A4_Score'] + dfTratado['A5_Score'] + dfTratado['A6_Score'] + dfTratado['A7_Score'] + dfTratado['A8_Score'] + dfTratado['A9_Score'] + dfTratado['A10_Score'] 
label_encoder = preprocessing.LabelEncoder()
dfTratado['ethnicity']= label_encoder.fit_transform(dfTratado['ethnicity']) 
dfTratado['contry_of_res']= label_encoder.fit_transform(dfTratado['contry_of_res']) 
dfTratado

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,Class/ASD,PuntuacionTest
0,1,0,1,0,1,0,1,0,1,1,38.172746,1,5,0,0,7,0,6.351166,0,6
1,0,0,0,0,0,0,0,0,0,0,47.750517,0,5,0,0,25,0,2.255185,0,0
2,1,1,1,1,1,1,1,1,1,1,7.380373,0,9,0,1,54,0,14.851484,1,10
3,0,0,0,0,0,0,0,0,0,0,23.561927,1,5,0,0,54,0,2.276617,0,0
4,0,0,0,0,0,0,0,0,0,0,43.205790,0,5,0,0,46,0,-4.777286,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,1,0,0,0,0,0,0,1,1,16.597187,0,2,0,0,36,0,12.999501,0,3
796,0,1,1,0,0,1,0,1,1,1,20.703001,0,9,0,0,17,0,13.561518,0,6
797,0,0,0,0,0,0,0,0,0,0,5.711481,0,7,1,0,36,0,2.653177,0,0
798,0,0,0,0,0,0,0,0,0,0,16.414305,1,5,0,0,15,0,9.069342,0,0


Para tener guardadas todas las transformaciones, creo un Pipeline que luego podré aplicar a otros conjuntos de datos. Además de las transformaciones que había hecho antes, voy a aprovechar para normalizar las columnas numéricas

In [41]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config

set_config(transform_output="pandas")

num_cols = ["age", "result", "PuntuacionTest"]

class DFTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
         X.drop(["ID", "age_desc", "relation"], axis=1, inplace=True)
         X['gender'] = X['gender'].map({'m': 0, 'f': 1})
         X['jaundice'] = X['jaundice'].map({'no': 0, 'yes': 1})
         X['austim'] = X['austim'].map({'no': 0, 'yes': 1})
         X['used_app_before'] = X['used_app_before'].map({'no': 0, 'yes': 1})
         X['ethnicity'] = X['ethnicity'].replace({'others': 'Others', '?': 'Others'})
         X['PuntuacionTest'] = X['A1_Score'] + X['A2_Score'] + X['A3_Score'] + X['A4_Score'] + X['A5_Score'] + X['A6_Score'] + X['A7_Score'] + X['A8_Score'] + X['A9_Score'] + X['A10_Score'] 
         label_encoder = preprocessing.LabelEncoder()
         X['ethnicity']= label_encoder.fit_transform(X['ethnicity']) 
         X['contry_of_res']= label_encoder.fit_transform(X['contry_of_res']) 

         scaler = StandardScaler()
         X[num_cols] = scaler.fit_transform(X[num_cols])

         return X


pipeline = Pipeline([

    ('tratarDF', DFTransformer())

])



Para probar el Pipeline, voy a abrir el archivo test.csv y lo voy a pasar por el Pipeline. Me guardaré el dataFrame resultante para utilizarlo al final de la práctica

In [42]:

dfTest = pd.read_csv('test.csv')
dfTest  = pipeline.fit_transform(dfTest)
dfTratado  = pipeline.fit_transform(dfInicial.copy())
dfTest

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,PuntuacionTest
0,1,1,0,0,1,1,0,0,1,1,-0.723383,0,9,1,0,14,0,0.793428,0.291998
1,1,0,0,0,0,0,0,1,0,0,0.076414,0,0,0,0,21,0,-0.451187,-0.836497
2,1,1,1,0,1,1,0,1,1,1,0.384605,0,9,1,0,10,0,-1.168682,0.856246
3,0,0,0,0,0,0,0,0,0,0,-0.048710,0,5,0,0,14,0,-1.372993,-1.400745
4,0,0,0,1,0,0,0,0,0,0,-1.173700,0,5,0,0,17,0,-0.302103,-1.118621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,0,0,1,0,0,1,1,1,-0.205455,0,1,0,0,4,0,-2.253420,0.291998
196,1,0,0,0,0,0,0,0,0,1,-0.838278,0,5,0,0,14,0,-1.734947,-0.836497
197,1,0,0,0,0,0,1,0,1,1,-0.228893,0,5,0,0,23,0,-1.100936,-0.272250
198,0,1,0,0,0,0,0,1,0,1,-0.118916,1,5,0,0,32,0,-0.654892,-0.554374


<font color=#FFFF00>5. CLASIFICACIÓN CON REGRESIÓN LOGÍSTICA. MÉTRICAS. PREDICCIONES.</font>

In [43]:
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score 


Vamos a dividir el dataframe en un 75% para train y un 25% para test, pero además lo vamos a hacer respetando la proporcion de filas que tienen el Class/ASD = 1 en cada conjunto

In [44]:
X_train, X_test = model_selection.train_test_split(dfTratado, stratify = dfTratado["Class/ASD"])

print("TRAIN", X_train.groupby("Class/ASD")["Class/ASD"].count())
print("TEST", X_test.groupby("Class/ASD")["Class/ASD"].count())


TRAIN Class/ASD
0    479
1    121
Name: Class/ASD, dtype: int64
TEST Class/ASD
0    160
1     40
Name: Class/ASD, dtype: int64


Ahora voy a ajustar el modelo de regresión logística con los datos de train

In [45]:
y_train = X_train['Class/ASD']
X_train = X_train.drop(["Class/ASD"], axis=1, inplace=False)

y_test = X_test['Class/ASD']
X_test = X_test.drop(["Class/ASD"], axis=1, inplace=False)

modelLR = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)
modelLR.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

Compruebo qué tal desempeño tien este algoritmo con los datos de train

In [46]:
modelLR.score(X_train,y_train)

0.885

Y a continuación lo compruebo con los datos de test

In [47]:
modelLR.score(X_test,y_test)

0.875